In [2]:
import os
import requests
import pandas as pd
import kaggle

In [3]:
kaggle.api.authenticate()

In [4]:
os.system('kaggle datasets download -d divyansh22/friends-tv-show-script --unzip --p "data/"')

 60%|█████▉    | 1.00M/1.67M [00:00<00:00, 3.44MB/s]

100%|██████████| 1.67M/1.67M [00:01<00:00, 1.74MB/s]


0